In [1]:
import sys, os
sys.path.append(os.getcwd())
print(sys.path)

['/Users/chrisg/miniconda3/envs/drlnd/lib/python36.zip', '/Users/chrisg/miniconda3/envs/drlnd/lib/python3.6', '/Users/chrisg/miniconda3/envs/drlnd/lib/python3.6/lib-dynload', '', '/Users/chrisg/miniconda3/envs/drlnd/lib/python3.6/site-packages', '/Users/chrisg/miniconda3/envs/drlnd/lib/python3.6/site-packages/IPython/extensions', '/Users/chrisg/.ipython', '/Users/chrisg/dev/deep-reinforcement-multiagent']


In [2]:
from unityagents import UnityEnvironment
import numpy as np

In [3]:
env = UnityEnvironment(file_name="Tennis.app", no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
brain_name

'TennisBrain'

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [6]:
### Random Actions

for i in range(1, 2):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        print('actions', actions)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        #print(actions)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        print('next_states', next_states)
        rewards = env_info.rewards                         # get reward (for each agent)
        print('rewards', rewards)
        dones = env_info.local_done                        # see if episode finished
        print('dones', dones)
        scores += env_info.rewards                         # update the score (for each agent)
        print('scores', scores)
        states = next_states                               # roll over states to next time step
        #if np.any(dones):                                  # exit loop if episode finished
        if True:
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))


actions [[1.3360123  0.2805893 ]
 [0.05604431 0.34346752]]
next_states [[ 0.          0.          0.          0.          0.          0.
   0.          0.         -7.38993645 -1.5        -0.          0.
   6.83172083  5.99607611 -0.          0.         -4.38993692 -1.55886006
  30.         -0.98100001  6.83172083  5.91759634 30.         -0.98100001]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.         -6.70024681 -1.5         0.          0.
  -6.83172083  5.99607611  0.          0.         -6.53211403 -1.55886006
   1.68132925 -0.98100001 -6.83172083  5.91759634  1.68132925 -0.98100001]]
rewards [0.0, 0.0]
dones [False, False]
scores [0. 0.]
Score (max over agents) from episode 1: 0.0


In [10]:
from ddpg_agent import Agent

agent = Agent(state_size, action_size, 11, 99)

print(agent.actor_local_0)
print(agent.critic_local_0)

Actor(
  (fc1): Linear(in_features=24, out_features=400, bias=True)
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=400, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=2, bias=True)
)
Critic(
  (fcs1): Linear(in_features=48, out_features=400, bias=True)
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=404, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=1, bias=True)
)


In [ ]:
from collections import deque
import matplotlib.pyplot as plt
import torch
from timeit import default_timer as timer

# DEFAULTS:
# n_episodes = 10000
# max_t = 1000
# print_every = 10
# target_window = 100
# target_score = 0.5
def run(n_episodes=10000, max_t=1000, print_every=10, target_window=100, target_score=0.5):
    scores_deque = deque(maxlen=target_window)
    scores_total = []
    train_every = 20
    train_iterations = 15
    time_deque = deque(maxlen=print_every)
    
    for i_episode in range(1, n_episodes+1):
        
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        scores = np.zeros(num_agents)
        agent.reset()
         
        start = timer()
        t_done = 0
        for t in range(max_t):
            
            # actions between -1,1 calls for tanh
            #print(states)
            actions = agent.act(states, t).cpu().detach().numpy()
            #print(actions)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            #print(next_states)
            rewards = env_info.rewards
            #print(rewards)
            dones = env_info.local_done
            #print(dones)
            
            train = not bool(t % train_every)
            
            agent.step(states, actions, rewards, next_states, dones, train, train_iterations)  
            
            states = next_states
            scores += rewards
            #print('r',rewards)
            #print('s',scores)
            if np.any(dones):
                t_done = t
                #print('done at', t_done)
                break
                
        end = timer()
        delta = end - start
        time_deque.append(delta)
        scores_max = np.max(scores)
        scores_deque.append(scores_max)
        scores_total.append(scores_max)
        scores_deque_mean = np.mean(scores_deque)
        print('\rEpisode {} t{}\tAverage Score: {:.4f}\t({:.0f} sec)'.format(i_episode, t_done, scores_deque_mean, delta), end="")
      
        #print(np.max(scores_deque))
        if i_episode % print_every == 0:
            #print(np.max(scores_deque))
            scores_deque_max = np.max(scores_deque)
            torch.save(agent.actor_local_0.state_dict(), 'checkpoint_actor_local_0.pth')
            torch.save(agent.actor_local_1.state_dict(), 'checkpoint_actor_local_1.pth')               
            print('\rEpisode {}\tAverage Score: {:.4f}\tMax: {:.4f}\t({:.0f} sec avrg)'\
                  .format(i_episode, scores_deque_mean, scores_deque_max, np.mean(time_deque)))
        sma_bool = [s >= target_score for s in scores_deque]
        #print(scores_deque)
        #if np.all(sma_bool) and len(sma_bool) >= target_window: 
        if scores_deque_mean >= target_score and len(scores_deque) >= target_window:
            print("\n*** Environment Solved ***")
            torch.save(agent.actor_local_0.state_dict(), 'env_actor_local_0.pth')
            torch.save(agent.actor_local_1.state_dict(), 'env_actor_local_1.pth')          
            break
                
            
    return scores_total, scores_deque

scores, sma = run()
print([ int(s*1000)/1000.0 for s in sma])
#print(scores)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.savefig('scores.png')            

In [ ]:
env.close()